In [23]:
import os
import numpy as np
import cv2
from pathlib import Path
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Concatenate,BatchNormalization,Activation
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras.callbacks import LearningRateScheduler
import keras.backend as K
import numpy as np
from keras.layers import LeakyReLU
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Model
#from skimage.measure import compare_psnr
import argparse
from pathlib import Path

epochs=2
batch_size=128

sigma=25.0  

coderootpath = 'C:\\Test\\DnCNN\\'
genDataPath=coderootpath + 'RawData\\'
save_dir=coderootpath + 'training\\'
data=coderootpath + 'training/img_clean_pats.npy'
datafinalpath = "C:\\Test\\DnCNN\\Stryde_input_data_csg1.npy"



In [21]:
#SisData = np.load(datafinalpath)

#higheit = SisData.shape[1]
#wideth = SisData.shape[2]
def prepare_data(SisData,pat_size=80, stride=10, noiselevel = 0.25):
    count = 0

    for i in range(len(SisData)):
        higheit,wideth,_ = SisData[i,:].shape
        for x in range(0,(higheit-pat_size),
                      stride):
            for y in range(0,(wideth-pat_size),
                      stride):
                count +=1
    print(count)
    inputs=np.ones((int(count), int(pat_size), int(pat_size),1),dtype=np.float)

    count = 0

    for imgindex in range(len(SisData)):
        datacache = SisData[imgindex,:]
        #datacache = (datacache - np.min(datacache))/(np.max(datacache) - np.min(datacache))
        im_h,im_w,_ = datacache.shape
        for x in range(0,im_h- pat_size, stride):
            for y in range(0,im_w- pat_size, stride):
                inputs[count,:,:,:]=datacache[x:x+pat_size, y:y+pat_size,:]
                count += 1


    noises = np.random.normal(0.0,noiselevel,inputs.shape)
    noisesinput = noises + inputs
    return noises, noisesinput

def downsample2(SisData):
    # low pass filter before downasample when have time. 
    if len(SisData.shape) >= 3:
        sensorsize = SisData.shape[1]
    else:
        sensorsize = SisData.shape[0]
    
    Indexes = range(0,sensorsize,2)
    
    if len(SisData.shape) == 4:
        SisDatadownsample = SisData[0,Indexes,:,:]
    else:
        SisDatadownsample = SisData[Indexes,:]
    return(SisDatadownsample)
    
def builddnCNN(kernalsize,layerlevel = 3):
    model=Sequential()

    model.add(Conv2D(8,(kernalsize[0,0],kernalsize[0,1]),padding="same",input_shape=(None,None,1)))
    model.add(LeakyReLU(0.1))

  
    for layers in range(1,layerlevel):
        if layers <= (kernalsize.shape[0]-2):
            model.add(Conv2D(18,(kernalsize[layers,0],kernalsize[layers,1]),padding="same"))
            print(kernalsize[layers,0], " + " , kernalsize[layers,1])
        else:
            model.add(Conv2D(16,(kernalsize[(kernalsize.shape[0]-2),0],kernalsize[(kernalsize.shape[0]-2),1]),padding="same"))
            print(kernalsize[(kernalsize.shape[0]-2),0], " + " , kernalsize[(kernalsize.shape[0]-2),1])

        model.add(BatchNormalization())
        model.add(LeakyReLU(0.1))
        #model.add(Activation('relu'))
    model.add(Conv2D(1,(kernalsize[(kernalsize.shape[0]-1),0],kernalsize[(kernalsize.shape[0]-1),1]),padding="same"))
    print(kernalsize[(kernalsize.shape[0]-1),0], " + " , kernalsize[(kernalsize.shape[0]-1),1])

    return(model)

def trainingModel(model, noisesinput, noises, savemodelname = "C:/Test/DnCNN/modelsaved.h5", epochs = 1):
    opt=optimizers.Adam(learning_rate=0.0005)
    model.compile(loss='mse',optimizer=opt)
    early_stopping = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=15)

    # train
    History = model.fit(x = noisesinput, y = noises, 
                        epochs=epochs, 
                        verbose=1, 
                        batch_size = 128,
                        validation_split = 0.3,
                        callbacks = [early_stopping])
    
    model.save(savemodelname)
    return(model, History)

In [14]:
# need to change the data loading and processing interface based on the data structures latter. 
# data is prepared with 80X80 format.
# only have one piece of data, 70% training, 30% validation, no testing. later will add certain portions for testing.

SisDataTrain = np.load(datafinalpath)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (-1,1))
scaler = scaler.fit(SisDataTrain)
SisDataTrain1 = scaler.transform(SisDataTrain)

SisDataTrain1 = np.reshape(SisDataTrain1,(1,3000,240,1))


#SisDataTrain = SisDataTrain / np.max([np.max(SisDataTrain),np.abs(np.min(SisDataTrain))])
noises, noisesinput = prepare_data(SisData = SisDataTrain1,pat_size=80, stride=15, noiselevel = 0.02)
noisesinput = np.clip(noisesinput, a_min = -1, a_max = 1)


SisDataTrain_D2 = downsample2(SisData = SisDataTrain1)
SisDataTrain_D2 = np.reshape(SisDataTrain_D2,(1,1500,240,1))
noisescache, noisesinputcache = prepare_data(SisData = SisDataTrain_D2,pat_size=80, stride=15, noiselevel = 0.02)
noisesinputcache = np.clip(noisesinputcache, a_min = -1, a_max = 1)
noises = np.append(noises,noisescache , axis=0)
noisesinput = np.append(noisesinput , noisesinputcache, axis=0)

SisDataTrain_D4 = downsample2(SisData = SisDataTrain_D2)
SisDataTrain_D4 = np.reshape(SisDataTrain_D4,(1,750,240,1))
noisescache, noisesinputcache = prepare_data(SisData = SisDataTrain_D4,pat_size=80, stride=15, noiselevel = 0.02)
noisesinputcache = np.clip(noisesinputcache, a_min = -1, a_max = 1)
noises = np.append(noises,noisescache, axis=0)
noisesinput = np.append(noisesinput , noisesinputcache, axis=0)

2145
1045
495


In [28]:
kernalsize = np.array([[4,3],[6,2],[8,2],[10,2],[3,3]])

dncnnmodel = builddnCNN(kernalsize = kernalsize,layerlevel = 15)
dncnnmodel.summary()

6  +  2
8  +  2
10  +  2
10  +  2
10  +  2
10  +  2
10  +  2
10  +  2
10  +  2
10  +  2
10  +  2
10  +  2
10  +  2
10  +  2
3  +  3
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, None, None, 8)     104       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, None, None, 8)     0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, None, None, 18)    1746      
_________________________________________________________________
batch_normalization_11 (Batc (None, None, None, 18)    72        
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, None, None, 18)    0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, None, None, 18)   

In [24]:
trainedmodel, traininghistory = trainingModel(model = dncnnmodel, noisesinput = noisesinput, noises = noises, savemodelname = "C:/Test/DnCNN/modelsaved.h5", epochs = 1)

21/21 [==============================] - 70s 3s/step - loss: 0.0504 - val_loss: 9.1400e-04


In [27]:
trainedmodel.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, None, None, 8)     104       
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, None, None, 8)     0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, None, None, 18)    1746      
_________________________________________________________________
batch_normalization_10 (Batc (None, None, None, 18)    72        
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, None, None, 18)    0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, None, None, 1)     163       
Total params: 2,085
Trainable params: 2,049
Non-trainable params: 36
___________________________________________________

In [ ]:
# validate and testing based on more data.